In [6]:
!unzip /content/train_set.zip

Streaming output truncated to the last 5000 lines.
  inflating: train_set/images/1004.jpg  
  inflating: train_set/images/1005.jpg  
  inflating: train_set/images/1006.jpg  
  inflating: train_set/images/1007.jpg  
  inflating: train_set/images/1008.jpg  
  inflating: train_set/images/1009.jpg  
  inflating: train_set/images/1010.jpg  
  inflating: train_set/images/1011.jpg  
  inflating: train_set/images/1012.jpg  
  inflating: train_set/images/1013.jpg  
  inflating: train_set/images/1014.jpg  
  inflating: train_set/images/1015.jpg  
  inflating: train_set/images/1016.jpg  
  inflating: train_set/images/1017.jpg  
  inflating: train_set/images/1018.jpg  
  inflating: train_set/images/1019.jpg  
  inflating: train_set/images/1020.jpg  
  inflating: train_set/images/1021.jpg  
  inflating: train_set/images/1022.jpg  
  inflating: train_set/images/1023.jpg  
  inflating: train_set/images/1024.jpg  
  inflating: train_set/images/1025.jpg  
  inflating: train_set/images/1026.jpg  
  infl

In [34]:
import numpy as np
import scipy.io
import sklearn.metrics
import sklearn 
import pandas as pd
import os
import sklearn.model_selection
import random
import tensorflow as tf


In [9]:
# read labels
labels = pd.read_csv('/content/train_set/label.csv')
y= labels['label'].to_numpy()

# read points
n = 3000
for i in range(1,n+1):
  p_path = str(i).zfill(4)+'.mat'
  points_path = os.path.join('/content/train_set/points/', p_path)
  mat = scipy.io.loadmat(points_path)
  if 'faceCoordinatesUnwarped' in mat:
    cords = mat['faceCoordinatesUnwarped'] 
  else:
    cords = mat['faceCoordinates2']

  distance = sklearn.metrics.pairwise_distances(cords)
  flatten_distance = distance[np.triu_indices(len(cords[:,0]), k = 1)]
  if i==1:
    distances = np.mat([flatten_distance])
  else:
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    

In [10]:
distances.shape

(3000, 3003)

In [11]:
# data preprocessing

print('The number of class 0 is ' + str(n-sum(y)))
print('The number of class 1 is ' + str(sum(y)))
print('Only %.2f'% (sum(y)/n*100) + '% of total dataset are class 1. ')
print('So, it is an unbalanced dataset, we need to do some data preprocessing.')
print('Here, we are using oversampling to generate more class 1 datasets.')

mat_1 = np.add(np.where(y == 1),1)
n_oversample = (n-sum(y))-sum(y)

for i in range(n_oversample):
  samples_index = random.sample(list(list(mat_1)[0]), 2)

  p_path = str(samples_index[0]).zfill(4)+'.mat'
  points_path = os.path.join('/content/train_set/points/', p_path)
  mat = scipy.io.loadmat(points_path)
  if 'faceCoordinatesUnwarped' in mat:
    cords_0 = mat['faceCoordinatesUnwarped'] 
  else:
    cords_0 = mat['faceCoordinates2']
  
  p_path = str(samples_index[1]).zfill(4)+'.mat'
  points_path = os.path.join('/content/train_set/points/', p_path)
  mat = scipy.io.loadmat(points_path)
  if 'faceCoordinatesUnwarped' in mat:
    cords_1 = mat['faceCoordinatesUnwarped'] 
  else:
    cords_1 = mat['faceCoordinates2']

  cords_new = (cords_0 + cords_1) / 2
  distance = sklearn.metrics.pairwise_distances(cords_new)
  flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
  distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
  y = np.append(y,np.array(1))


The number of class 0 is 2402
The number of class 1 is 598
Only 19.93% of total dataset are class 1. 
So, it is an unbalanced dataset, we need to do some data preprocessing.
Here, we are using oversampling to generate more class 1 datasets.


In [12]:
distances.shape,y.shape

((4804, 3003), (4804,))

In [ ]:
# import csv
# wtr = csv.writer(open ('labels_full.csv', 'w'), delimiter=',', lineterminator='\n')
# for x in y : wtr.writerow ([x])


In [ ]:
# a = np.asarray(distances)
# np.savetxt("dataset_balanced.csv", a, delimiter=",")


In [ ]:
# distances = pd.read_csv('/content/dataset_balanced.csv')
# y = pd.read_csv('/content/labels_full.csv')

In [ ]:
# distances = np.mat(distances.values)
# y = np.mat(y.values)

In [13]:
# distances.shape,y.shape

((4804, 3003), (4804,))

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(distances)
distances_scale = scaler.transform(distances)

In [15]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(distances_scale, y,
                                                                            test_size=0.2,
                                                                            random_state=123)

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import matplotlib.pylab as plt
from lightgbm import LGBMClassifier
import time

Logistic Regression Only

In [10]:
start_time = time.time()
lr = LogisticRegression(random_state=123, C=0.01, max_iter=300, multi_class='multinomial' ).fit(X_train,y_train)
end_time = time.time()
print("Training the Logistics Regression uses %s seconds" % (end_time-start_time))

Training the Logistics Regression uses 15.447359561920166 seconds


In [11]:
y_fit = lr.predict(X_train)
y_fitprob = lr.predict_proba(X_train)[:,1]
y_pred = lr.predict(X_test)
y_predprob = lr.predict_proba(X_test)[:,1]
sklearn.metrics.accuracy_score(y_train, y_fit), sklearn.metrics.roc_auc_score(y_train, y_fitprob), sklearn.metrics.accuracy_score(y_test,y_pred), sklearn.metrics.roc_auc_score(y_test, y_predprob)

(0.8683320322664585,
 0.9380964763305516,
 0.8532778355879292,
 0.9159439394990633)

GBDT Only

In [28]:
gbdt = GradientBoostingClassifier(random_state=123)
gbdt.fit(X_train,y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=123, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [29]:
y_fit = gbdt.predict(X_train)
y_fitprob = gbdt.predict_proba(X_train)[:,1]
y_pred = gbdt.predict(X_test)
y_predprob = gbdt.predict_proba(X_test)[:,1]
sklearn.metrics.accuracy_score(y_train, y_fit), sklearn.metrics.roc_auc_score(y_train, y_fitprob), sklearn.metrics.accuracy_score(y_test,y_pred), sklearn.metrics.roc_auc_score(y_test, y_predprob)

(0.9523809523809523, 0.9910780115333881, 0.8470343392299687, 0.925280129050163)

In [19]:
gbdt = GradientBoostingClassifier(random_state=123, max_depth = 7, min_samples_split = 200,
                                  subsample = 0.85, max_features = 13, learning_rate = 0.1,
                                  n_estimators = 300)
gbdt.fit(X_train,y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=7,
                           max_features=13, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=200,
                           min_weight_fraction_leaf=0.0, n_estimators=300,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=123, subsample=0.85, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [20]:
y_fit = gbdt.predict(X_train)
y_fitprob = gbdt.predict_proba(X_train)[:,1]
y_pred = gbdt.predict(X_test)
y_predprob = gbdt.predict_proba(X_test)[:,1]
sklearn.metrics.accuracy_score(y_train, y_fit), sklearn.metrics.roc_auc_score(y_train, y_fitprob), sklearn.metrics.accuracy_score(y_test,y_pred), sklearn.metrics.roc_auc_score(y_test, y_predprob)

(1.0, 1.0, 0.8636836628511967, 0.9364679456046624)

In [42]:
param_test1  = {  
    'n_estimators':range(100,501,100),
    'learning_rate': [0.001,0.01,0.1,0.5]
}

gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(random_state=123,min_samples_split = 300,
                                  max_depth = 8 ,subsample = 0.8, max_features = 'sqrt'),
                        param_grid = param_test1, scoring='roc_auc',cv=3)
gsearch1.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=8,
                                                  max_features='sqrt',
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=300,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
     

In [43]:
gsearch1.best_params_, gsearch1.best_score_

({'learning_rate': 0.1, 'n_estimators': 500}, 0.944278946555829)

In [50]:
y_fit = gsearch1.predict(X_train)
y_fitprob = gsearch1.predict_proba(X_train)[:,1]
y_pred = gsearch1.predict(X_test)
y_predprob = gsearch1.predict_proba(X_test)[:,1]
sklearn.metrics.accuracy_score(y_train, y_fit), sklearn.metrics.roc_auc_score(y_train, y_fitprob), sklearn.metrics.accuracy_score(y_test,y_pred), sklearn.metrics.roc_auc_score(y_test, y_predprob)

(1.0, 1.0, 0.8803329864724245, 0.9526165614375911)

In [17]:
param_test2  = {  
    'max_depth':range(5,14,2),
    'min_samples_split': range(100,501,100)
}

gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(random_state=123, n_estimators = 500, 
                                  subsample = 0.8, max_features = 'sqrt', learning_rate = 0.1),
                        param_grid = param_test2, scoring='roc_auc',cv=3)
gsearch2.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features='sqrt',
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=500,
                                                  n_iter_no_change=None,
       

In [18]:
gsearch2.best_params_, gsearch2.best_score_

({'max_depth': 11, 'min_samples_split': 200}, 0.9453475049364958)

In [21]:
y_fit = gsearch2.predict(X_train)
y_fitprob = gsearch2.predict_proba(X_train)[:,1]
y_pred = gsearch2.predict(X_test)
y_predprob = gsearch2.predict_proba(X_test)[:,1]
sklearn.metrics.accuracy_score(y_train, y_fit), sklearn.metrics.roc_auc_score(y_train, y_fitprob), sklearn.metrics.accuracy_score(y_test,y_pred), sklearn.metrics.roc_auc_score(y_test, y_predprob)

(1.0, 1.0, 0.8834547346514048, 0.9571003607854021)

In [22]:
param_test3  = {  
     'min_samples_leaf':range(10,91,20),
     'subsample':[0.6,0.7,0.8,0.9]
}

gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(random_state=123, n_estimators = 500, 
                                                               max_depth = 11, min_samples_split = 200,
                                                               max_features = 'sqrt', learning_rate = 0.1),
                        param_grid = param_test3, scoring='roc_auc',cv=3)
gsearch3.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=11,
                                                  max_features='sqrt',
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=200,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=500,
                                                  n_iter_no_change=None,
    

In [23]:
gsearch3.best_params_, gsearch3.best_score_

({'min_samples_leaf': 10, 'subsample': 0.8}, 0.9462746723223064)

In [24]:
y_fit = gsearch2.predict(X_train)
y_fitprob = gsearch2.predict_proba(X_train)[:,1]
y_pred = gsearch2.predict(X_test)
y_predprob = gsearch2.predict_proba(X_test)[:,1]
sklearn.metrics.accuracy_score(y_train, y_fit), sklearn.metrics.roc_auc_score(y_train, y_fitprob), sklearn.metrics.accuracy_score(y_test,y_pred), sklearn.metrics.roc_auc_score(y_test, y_predprob)

(1.0, 1.0, 0.8834547346514048, 0.9571003607854021)

In [28]:
GBDT_best = GradientBoostingClassifier(random_state=123, n_estimators = 500, max_depth = 11, min_samples_split = 200,max_features = 'sqrt', min_samples_leaf=10, subsample=0.8, learning_rate = 0.1)

In [32]:
GBDT_best.fit(X_train,y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=11,
                           max_features='sqrt', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=200,
                           min_weight_fraction_leaf=0.0, n_estimators=500,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=123, subsample=0.8, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [31]:
y_fit = GBDT_best.predict(X_train)
y_fitprob = GBDT_best.predict_proba(X_train)[:,1]
y_pred = GBDT_best.predict(X_test)
y_predprob = GBDT_best.predict_proba(X_test)[:,1]
sklearn.metrics.accuracy_score(y_train, y_fit), sklearn.metrics.roc_auc_score(y_train, y_fitprob), sklearn.metrics.accuracy_score(y_test,y_pred), sklearn.metrics.roc_auc_score(y_test, y_predprob)

(1.0, 1.0, 0.8917793964620188, 0.955834142787761)

In [35]:
predict_one_hot_train = tf.one_hot(y_fit, depth=2)
predict_one_hot_test = tf.one_hot(y_pred, depth=2)

In [48]:
lr_after_gbdt = LogisticRegression(penalty='l2',random_state=123)
lr_after_gbdt.fit(predict_one_hot_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
y_fit = lr_after_gbdt.predict(predict_one_hot_train)
y_fitprob = lr_after_gbdt.predict_proba(predict_one_hot_train)[:,1]
y_pred = lr_after_gbdt.predict(predict_one_hot_test)
y_predprob = lr_after_gbdt.predict_proba(predict_one_hot_test)[:,1]
sklearn.metrics.accuracy_score(y_train, y_fit), sklearn.metrics.roc_auc_score(y_train, y_fitprob), sklearn.metrics.accuracy_score(y_test,y_pred), sklearn.metrics.roc_auc_score(y_test, y_predprob)

(1.0, 1.0, 0.8917793964620188, 0.8915800839519877)